In [1]:
import os, sys
import django
import pytz
from dateutil import parser
TIME_ZONE = 'Africa/Gaborone'
tz = pytz.timezone(TIME_ZONE)

sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "tshilo_dikotla.settings"

# Connect to Django ORM
django.setup()
import uuid
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

try:
    td = Site.objects.get(id=40)
except Site.DoesNotExist:
    td = Site.objects.create(id=40, domain='td-live.bhp.org.bw', name='td-live.bhp.org.bw')

In [2]:
# Subject Screening
from td_maternal.models import SubjectScreening
from td_maternal.identifiers import ScreeningIdentifier
from django.contrib.sites.models import Site

fname = '/Users/coulsonkgathi/source/td_data_files/20190609/maternal_eligibility.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sc_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        raise ValidationError('Line is not the right size')
    sc_data.append(data)

site = Site.objects.get(id=40)
count = 0
already_created = 0
for sc in sc_data:
    del sc['_state']
    del sc['registered_subject_id']
    sc.update(
        report_datetime=parser.parse(sc.get('report_datetime')).replace(tzinfo=tz),
        created=parser.parse(sc.get('created')).replace(tzinfo=tz),
        site=site,
        id=uuid.UUID(sc.get('id'))
    )
    del sc['eligibility_id']
    del sc['export_change_type']
    del sc['export_uuid']
    del sc['exported']
    del sc['exported_datetime']
    if sc['modified']:
        sc.update(modified=parser.parse(sc.get('modified')).replace(tzinfo=tz))
    else:
        sc.update(modified=None)
    try:
        SubjectScreening.objects.get(id=sc.get('id'))
    except SubjectScreening.DoesNotExist:
        sc.update(screening_identifier=ScreeningIdentifier().identifier)
        subject_Screening = SubjectScreening(**sc)
        subject_Screening.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created subject screening {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sc_data))
print("Existing: ", SubjectScreening.objects.all().count())

Total created subject screening 555
Already created 0
Expected:  555
Existing:  555


In [3]:
# TD Consent Version
import uuid
from td_maternal.models import TdConsentVersion
from django.core.exceptions import ValidationError

fname = '/Users/coulsonkgathi/source/td_data_files/20190609/td_consent_version.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

td_consent_version_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    td_consent_version_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for dd in td_consent_version_data:
    del dd['_state']
    created = parser.parse(dd.get('created'))
    created = created.replace(tzinfo=tz)
    dd.update(
        created=created,
        report_datetime=parser.parse(dd.get('report_datetime')).replace(tzinfo=tz),
        id=uuid.UUID(dd.get('id'))
    )
    maternal_eligibility_id = uuid.UUID(dd.get('maternal_eligibility_id'))
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        modified = modified.replace(tzinfo=tz)
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    try:
        subject_screening = SubjectScreening.objects.get(id=maternal_eligibility_id)
    except SubjectScreening.DoesNotExist:
        raise ValidationError('Subject Screening can not be missing ', maternal_eligibility_id)
    else:
        del dd['maternal_eligibility_id']
        screening_identifier = subject_screening.screening_identifier
        try:
            TdConsentVersion.objects.get(screening_identifier=screening_identifier)
        except TdConsentVersion.DoesNotExist:
            dd.update(screening_identifier=screening_identifier)
            td_consent_version = TdConsentVersion(**dd)
            td_consent_version.save_base(raw=True)
            count += 1
        else:
            already_created += 1
print(f'Total created td_consent_version {count}')
print(f'Already created td_consent_version {already_created}')

Total created td_consent_version 455
Already created td_consent_version 0


In [20]:
# Subject Consent
import uuid
from td_maternal.models import SubjectConsent
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

fname = '/Users/coulsonkgathi/source/td_data_files/20190609/maternal_consent.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

s_consent_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    s_consent_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for dd in s_consent_data:
    del dd['_state']
    del dd['export_change_type']
    del dd['export_uuid']
    del dd['exported']
    del dd['exported_datetime']
    del dd['study_site']
    dd.update(site=td)
    created = parser.parse(dd.get('created'))
    created = created.replace(tzinfo=tz)
    consent_datetime = parser.parse(dd.get('consent_datetime'))
    consent_datetime = consent_datetime.replace(tzinfo=tz)
    dob = parser.parse(dd.get('dob')).replace(tzinfo=tz)
    dob = dob.date()
    dd.update(
        created=created,
        consent_datetime=consent_datetime,
        report_datetime=parser.parse(dd.get('consent_datetime')).replace(tzinfo=tz),
        dob=dob,
        id=uuid.UUID(dd.get('id'))
    )
    if dd['subject_identifier_as_pk']:
        dd.update(
            subject_identifier_as_pk=uuid.UUID(dd.get('subject_identifier_as_pk')))
    else:
        dd.update(
            subject_identifier_as_pk=dd.get('id'))
    if dd['is_verified_datetime']:
        is_verified_datetime = parser.parse(dd.get('is_verified_datetime'))
        is_verified_datetime = is_verified_datetime.replace(tzinfo=tz)
        dd.update(
        is_verified_datetime=is_verified_datetime)
    else:
        dd.update(is_verified_datetime=None)
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        modified = modified.replace(tzinfo=tz)
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    if dd['is_verified'] == 'TRUE':
        dd.update(is_verified=True)
    elif dd['is_verified'] == 'FALSE':
        dd.update(is_verified=False)
    if dd['guardian_name']:
        guardian_name = dd['guardian_name']
        name1, name2 = guardian_name.split(',')
        guardian_name = name1 + ', ' + name2
        dd.update(guardian_name=guardian_name)
    try:
        subject_screening = SubjectScreening.objects.get(id=dd['maternal_eligibility_id'])
    except SubjectScreening.DoesNotExist:
        raise ValidationError('Eligibility can not be missing for this subject', dd['subject_identifier'])
    else:
        dd.update(screening_identifier=subject_screening.screening_identifier)
        del dd['maternal_eligibility_id']
    try:
        SubjectConsent.objects.get(subject_identifier=dd.get('subject_identifier'))
    except SubjectConsent.DoesNotExist:
        subject_consent = SubjectConsent(**dd)
#         subject_consent.save_base(raw=True)
        count += 1
        subject_screening.subject_identifier = dd['subject_identifier']
#         subject_screening.save_base(raw=True)
    else:
        already_created += 1
print(f'Total created consents {count}')
print(f'Already created {already_created}')

Total created consents 904
Already created 0


In [21]:
dd

{'assessment_score': 'Yes',
 'citizen': 'Yes',
 'comment': '',
 'confirm_identity': '809626822',
 'consent_copy': 'Yes',
 'consent_datetime': datetime.datetime(2017, 8, 8, 10, 31, tzinfo=<DstTzInfo 'Africa/Gaborone' LMT+2:10:00 STD>),
 'consent_reviewed': 'Yes',
 'consent_signature': 'Yes',
 'created': datetime.datetime(2017, 8, 8, 14, 30, 52, 298959, tzinfo=<DstTzInfo 'Africa/Gaborone' LMT+2:10:00 STD>),
 'dm_comment': '',
 'dob': datetime.date(1994, 10, 19),
 'first_name': 'PENCY TEBOGO',
 'gender': '',
 'guardian_name': '',
 'hostname_created': 'td.bhp.org.bw',
 'hostname_modified': 'td.bhp.org.bw',
 'id': UUID('ffb9e9aa-ab74-4dae-ae1b-231b611c09e4'),
 'identity': '809626822',
 'identity_type': 'OMANG',
 'initials': 'PTL',
 'is_dob_estimated': '-',
 'is_incarcerated': '-',
 'is_literate': 'Yes',
 'is_verified': 'True',
 'is_verified_datetime': datetime.datetime(2017, 10, 30, 12, 18, 14, 888185, tzinfo=<DstTzInfo 'Africa/Gaborone' LMT+2:10:00 STD>),
 'language': 'tn',
 'last_name': '